<a href="https://colab.research.google.com/github/zehavitc/NLP/blob/master/NLP_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HW3**

Matan Sabag 301388567

Zehavit Leibovich 305226391

Part 1

In [0]:
# imports

import matplotlib.pyplot as plt
import numpy as np
from gensim.models import Word2Vec
import gensim.downloader as api

In [0]:
def load(model_name):
  '''
  Downloading and loading model into memory, as a dictionary of arrays, the keys are the words.
  '''
  wv_from_bin = api.load(model_name)
  vocab = list(wv_from_bin.vocab.keys())
  print("Loaded vocab size %i" % len(vocab))
  return wv_from_bin
  

Part 2 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
analogy_file_path = "/content/drive/My Drive/NLP/HW3/analogy.txt"

In [0]:
from gensim.test.utils import datapath
import pandas as pd
import gc


groups_names = ["capital-world", "currency", "city-in-state", "family", "gram1-adjective-to-adverb", "gram2-opposite", "gram3-comparative", "gram6-nationality-adjective"]
models_names = ["glove-wiki-gigaword-300","glove-wiki-gigaword-50","word2vec-google-news-300", "glove-twitter-200", "fasttext-wiki-news-subwords-300"]

results = pd.DataFrame(columns = groups_names + ["Overall"])
for model_name in models_names:
  model = load(model_name)
  analogy_score, sections = model.wv.evaluate_word_analogies(datapath(analogy_file_path))
  results.loc[model_name, "Overall"] = analogy_score
  for section in sections:
    group_name = section['section']
    if group_name in groups_names:
      results.loc[model_name, group_name] =  len(section['correct']) / (len(section['correct']) + len(section['incorrect']))
  del model
  gc.collect()
  
results
  

Loaded vocab size 400000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Loaded vocab size 400000
Loaded vocab size 3000000
Loaded vocab size 1193514


Part 3

In [0]:
def show_embeddings(data, labels):
    plt.scatter(data[:, 0], data[:, 1], marker='o')

    for label, x, y in zip(labels, data[:, 0], data[:, 1]):
        plt.annotate(
            label,
            xy=(x, y), xytext=(-20, 20),
            textcoords='offset points', ha='right', va='bottom',
            bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.2),
            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

# Testing        
show_embeddings(np.array([[1, 2],[2, 3]]), ["a","b"])

In [0]:
from sklearn.decomposition import TruncatedSVD

def reduce_dim(model, number_of_vectors=10000, required_words=["car"]):
  '''
  number_of_vectors: you should first truncate your model to only this number of vectors. Make sure to shuffle the model first, so that you choose random vectors.
  required_words: in addition to the number_of_vectors vectors you keep, you should keep the vectors of these words.
  Return the new, reduced model.
  '''
  words = list(model.vocab.keys())
  np.random.shuffle(words)
  X = []
  for word in (words[:number_of_vectors] + required_words):
    X.append(model.get_vector(word))
    
  X = np.asarray(X)
  reducer  = TruncatedSVD(n_components = 2)
  reducer.fit(X)
  return reducer.transform(X)
  

In [0]:
required_words = ["aircraft", "ambulance", "bicycle", "bike", "boat", "car", "truck", "chopper", "canoe", "driver", "lorry", "suv", "train",
                  "vehicle", "yacht", "apple", "banana", "orange", "tomato", "pepper", "watermelon", "grapes", "fruit", "fruits", "vegetables"]
model = load("glove-wiki-gigaword-50")
reduced_model = reduce_dim(model, number_of_vectors=10000, required_words=required_words)
show_embeddings(reduced_model[10000:], required_words)
del model

Part 4 

In [0]:
from gensim.models import Word2Vec
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown_w2v = Word2Vec(brown.sents())

In [0]:
def get_term_sentences(corpus, term):
  res = []
  for sentence in corpus.sents():
    str_sent = ' '.join(sentence)
    lowered_sent = str_sent.lower() 
    if term in lowered_sent:
      res.append(str_sent)
  return res
  

In [0]:
def get_word_vector(model_w2v, word):  
  if word in model_w2v:
    return model_w2v[word]
  
  if word.lower() in model_w2v:
    return model_w2v[word.lower()]
  
  return None

In [0]:
def get_neighbour_words(sentences, term, take_before):
  res = []
  for sentence in sentences:
    words = sentence.split()
    term_idx = [i for i, word in enumerate(words) if word == term]
    for idx in term_idx:
      if take_before:
        if idx > 0:
          res.append(words[idx - 1])
        if idx > 1:
          res.append(words[idx - 2])
      elif not(take_before):
        if idx < len(words) - 1:
          res.append(words[idx+1])
        if idx < len(words) - 2:
          res.append(words[idx + 2])
  return res

In [0]:
import numpy as np

terms = ["new york", "united states"]
for term in terms:
  term_words = term.split()
  sentences = get_term_sentences(brown, term)
  neighbours = get_neighbour_words(sentences, term_words[0], True) + get_neighbour_words(sentences, term_words[1], False)
  neighbours_vectors = []
  for word in neighbours:
    word_vec = get_word_vector(brown_w2v, word)
    if word_vec is not None:
      neighbours_vectors.append(word_vec)
    
  term_vec = np.array(neighbours_vectors)
  print(f'{term} vector: {np.mean(term_vec, axis=0)}')